<a href="https://colab.research.google.com/github/Hiromi06/machine-translation/blob/main/Marian_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from tqdm import tqdm
from transformers import MarianTokenizer


In [2]:
# Function to load text data
def load_data(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            lines.append(line.strip())
    return lines

# File paths
en_train_path = '/content/drive/MyDrive/machine_learning/JP_en_train.txt'
en_test_path = '/content/drive/MyDrive/machine_learning/JP_en_test.txt'
ja_train_path = '/content/drive/MyDrive/machine_learning/JP_ja_train.txt'
ja_test_path = '/content/drive/MyDrive/machine_learning/JP_ja_test.txt'

# Load data
en_train = load_data(en_train_path)
en_test = load_data(en_test_path)
ja_train = load_data(ja_train_path)
ja_test = load_data(ja_test_path)

In [3]:
# Tokenizer
model_name = 'Helsinki-NLP/opus-mt-ja-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/782k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
# Tokenize and save data in chunks
def tokenize_and_save_chunks(texts, tokenizer, prefix, output_dir, num_chunks=10, max_length=512):
    chunk_size = len(texts) // num_chunks
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(texts)
        chunk_texts = texts[start_idx:end_idx]

        tokenized_data = {
            'input_ids': [],
            'attention_mask': []
        }

        for text in tqdm(chunk_texts, desc=f"Tokenizing chunk {i + 1}/{num_chunks}"):
            tokenized = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length)
            tokenized_data['input_ids'].append(tokenized['input_ids'].squeeze(0))
            tokenized_data['attention_mask'].append(tokenized['attention_mask'].squeeze(0))

        # Convert lists to tensors
        tokenized_data['input_ids'] = torch.stack(tokenized_data['input_ids'])
        tokenized_data['attention_mask'] = torch.stack(tokenized_data['attention_mask'])

        # Save chunk
        file_path = os.path.join(output_dir, f'{prefix}_encoded_chunk_{i + 1}.pt')
        torch.save(tokenized_data, file_path)

# Define output directory
output_dir = '/content/drive/MyDrive/machine_learning/MarianMT/Marian_encoded_data'

# Tokenize and save each dataset
tokenize_and_save_chunks(en_train, tokenizer, 'en_train', output_dir)
tokenize_and_save_chunks(en_test, tokenizer, 'en_test', output_dir)
tokenize_and_save_chunks(ja_train, tokenizer, 'ja_train', output_dir)
tokenize_and_save_chunks(ja_test, tokenizer, 'ja_test', output_dir)

print("Data has been tokenized, split into chunks, and saved successfully.")



Tokenizing chunk 10/10: 100%|██████████| 87640/87640 [00:38<00:00, 2262.44it/s]


Data has been tokenized, split into chunks, and saved successfully.
